# **Link Prediction of Graph**

## **Installation**

In [1]:
!pip install torch_geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 467 kB 8.4 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.1.0.post1-py3-none-any.whl size=689859 sha256=db619c0c13f7c0c40ba94896ff345d187f3715fd8a5fb1c7c2fca249bea526ac
  Stored in directory: /root/.cache/pip/wheels/d1/cb/43/f7f2e472de4d7cff31bceddadc36d634e1e545fbc17961c282
Successfully built torch-geometric


In [2]:
!pip install torch_sparse

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 32.2 MB/s 
  Created wheel for torch-sparse: filename=torch_sparse-0.6.15-cp37-cp37m-linux_x86_64.whl size=516860 sha256=20596c6cd4d54a2ace5805eb9404445b9a8928b0e2f014f2f149ebee7a85fa7e
  Stored in directory: /root/.cache/pip/wheels/15/68/4d/1414be5c2c622bad35364e13213180797717b6d4b8923936dc
Successfully built torch-sparse


In [3]:
!pip install torch_scatter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for torch-scatter: filename=torch_scatter-2.0.9-cp37-cp37m-linux_x86_64.whl size=274491 sha256=c961cc627e4da3ff97894be7394dcd58a8e76c496acff5156feda7b7c2e8da4b
  Stored in directory: /root/.cache/pip/wheels/dd/57/a3/42ea193b77378ce634eb9454c9bc1e3163f3b482a35cdee4d1
Successfully built torch-scatter


In [4]:
import os.path as osp

import torch
from sklearn.metrics import roc_auc_score

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## **Data Transformation**

In [6]:
transform = T.Compose([
    T.NormalizeFeatures(),
    T.ToDevice(device),
    T.RandomLinkSplit(num_val=0.05, num_test=0.1, is_undirected=True,
                      add_negative_train_samples=False),
])

In [7]:
dataset = 'Cora'
path = osp.join('.', 'data', dataset)
#path = osp.join(osp.dirname(osp.realpath(__file__)), '..', 'data', 'Planetoid')
dataset = Planetoid(path, name='Cora', transform=transform)

Processing...
Done!


In [8]:
# After applying the `RandomLinkSplit` transform, the data is transformed from
# a data object to a list of tuples (train_data, val_data, test_data), with
# each element representing the corresponding split.
train_data, val_data, test_data = dataset[0]

## **Data Modeling**

In [9]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()

In [10]:
model = Net(dataset.num_features, 128, 64).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()

In [11]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)

    # We perform a new round of negative sampling for every training epoch:
    neg_edge_index = negative_sampling(
        edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
        num_neg_samples=train_data.edge_label_index.size(1), method='sparse')

    edge_label_index = torch.cat(
        [train_data.edge_label_index, neg_edge_index],
        dim=-1,
    )
    edge_label = torch.cat([
        train_data.edge_label,
        train_data.edge_label.new_zeros(neg_edge_index.size(1))
    ], dim=0)

    out = model.decode(z, edge_label_index).view(-1)
    loss = criterion(out, edge_label)
    loss.backward()
    optimizer.step()
    return loss

## **Model Evaluation**

In [12]:
@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())


best_val_auc = final_test_auc = 0
for epoch in range(1, 101):
    loss = train()
    val_auc = test(val_data)
    test_auc = test(test_data)
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        final_test_auc = test_auc
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f}, '
          f'Test: {test_auc:.4f}')

Epoch: 001, Loss: 0.6930, Val: 0.6896, Test: 0.6975
Epoch: 002, Loss: 0.6832, Val: 0.6812, Test: 0.6901
Epoch: 003, Loss: 0.7077, Val: 0.6866, Test: 0.6992
Epoch: 004, Loss: 0.6777, Val: 0.7101, Test: 0.7213
Epoch: 005, Loss: 0.6853, Val: 0.7336, Test: 0.7411
Epoch: 006, Loss: 0.6883, Val: 0.7444, Test: 0.7522
Epoch: 007, Loss: 0.6890, Val: 0.7400, Test: 0.7482
Epoch: 008, Loss: 0.6881, Val: 0.7235, Test: 0.7340
Epoch: 009, Loss: 0.6849, Val: 0.7044, Test: 0.7196
Epoch: 010, Loss: 0.6794, Val: 0.6911, Test: 0.7105
Epoch: 011, Loss: 0.6730, Val: 0.6859, Test: 0.7057
Epoch: 012, Loss: 0.6766, Val: 0.6879, Test: 0.7056
Epoch: 013, Loss: 0.6735, Val: 0.6921, Test: 0.7068
Epoch: 014, Loss: 0.6649, Val: 0.6994, Test: 0.7107
Epoch: 015, Loss: 0.6622, Val: 0.7101, Test: 0.7161
Epoch: 016, Loss: 0.6592, Val: 0.7181, Test: 0.7196
Epoch: 017, Loss: 0.6548, Val: 0.7173, Test: 0.7181
Epoch: 018, Loss: 0.6462, Val: 0.7147, Test: 0.7154
Epoch: 019, Loss: 0.6383, Val: 0.7276, Test: 0.7225
Epoch: 020, 

In [13]:
print(f'Final Test: {final_test_auc:.4f}')

z = model.encode(test_data.x, test_data.edge_index)
final_edge_index = model.decode_all(z)

Final Test: 0.9047


## **References**



*   https://github.com/pyg-team/pytorch_geometric/blob/master/examples/link_pred.py







